spreadsheet.ipynb

## Specifiche da implementare in Erlang
 Un worksheet deve essere una “matrice” di NxM celle ;
 Una cella può contenere qualsiasi tipo di dato primitivo o  il valore 'undef' .
 Il modulo del worksheet condiviso deve prevedere alcune funzioni di interfaccia sintatticamente definite
 1) new(name) -> spreadsheet | {error,reason} % Crea un nuovo foglio di nome “name” di dimensioni NxM di K tab (fogli)
  assegna il processo creatore come proprietario del foglio. I parametri N, M, K sono default nel modulo
 2) new(name, N, M, K) -> spreadsheet | {error,reason} % Crea un nuovo foglio di nome “name” di K fogli in cui ogni tab ha dimensioni NxM • assegna il processo creatore come proprietario del foglio

In [ ]:
-module(spreadsheet).
-export([new/1, new/4]).

-record(spreadsheet, {
    name,
    tabs = [],         % Lista di fogli (ogni foglio è una matrice NxM)
    owner = undefined  % Il proprietario del foglio
}).

% Funzione per creare un nuovo foglio con dimensioni default
new(Name) ->
    N = 10,  % Default N (numero di righe)
    M = 10,  % Default M (numero di colonne)
    K = 1,   % Default K (numero di tab)
    new(Name, N, M, K).

% Funzione per creare un nuovo foglio con dimensioni specificate
new(Name, N, M, K) when is_integer(N), is_integer(M), is_integer(K), N > 0, M > 0, K > 0 ->
    % Verifica se il nome del foglio esiste già
    case whereis(Name) of
        undefined ->
            % Creazione del processo proprietario
            Owner = self(),
            % Creazione dei tab (K fogli, ciascuno una matrice NxM)
            Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
            % Creazione del record spreadsheet
            Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
            % Registrazione del processo con il nome del foglio
            register(Name, self()),
            % Memorizzazione dello stato
            loop(Spreadsheet);
        _ ->
            {error, already_exists}
    end;
new(_, _, _, _) ->
    {error, invalid_parameters}.

% Funzione per creare un tab (una matrice NxM di celle)
create_tab(N, M) ->
    lists:map(fun(_) -> lists:duplicate(M, undef) end, lists:seq(1, N)).

% Loop principale del processo, dove si gestiscono i messaggi
loop(Spreadsheet) ->
    receive
        % Gestire messaggi per operazioni sul foglio di calcolo
        stop ->
            ok; % Fermare il processo
        _Other ->
            loop(Spreadsheet)
    end.


## Loop/1 e Loop/5(= diventerà starter/5)

**PROBLEMA**:se il pid del processo Spreadsheet coincide con self() per far sì che  coincida con il processo che lo ha generato (implementazione del requisito di proprietà), la shell resta in ascolto di messaggi senza fornire prompt.

**SOLUZIONE**:modello basato su processi separati, dove il processo che gestisce lo stato del foglio di calcolo (Spreadsheet) viene creato come un processo figlio separato dal processo proprietario.Proprietario: Il processo che crea e controlla il foglio di calcolo.
Processo separato per Spreadsheet: Il processo Spreadsheet deve essere separato dal proprietario usando la funzione spawn/1 o spawn/3 per creare un processo separato che gestisce il ciclo loop/1 del foglio di calcolo

In [ ]:
-module(spreadsheet).
-export([new/1, new/4, read_cell/3, write_cell/4]).

-record(spreadsheet, {
    name,
    tabs = [],
    owner = undefined,
    access_policies = []
}).

% Funzione per creare un nuovo foglio con dimensioni default (separato dal proprietario)
new(Name) ->
    N = 10,
    M = 10,
    K = 1,
    new(Name, N, M, K).

% Funzione per creare un nuovo foglio di calcolo con un processo separato
new(Name, N, M, K) when is_integer(N), is_integer(M), is_integer(K), N > 0, M > 0, K > 0 ->
    case whereis(Name) of
        undefined ->
            Owner = self(),  % Il proprietario è il processo che chiama new/4
            Pid = spawn(spreadsheet, loop, [Name, Owner, N, M, K]),  % Crea il processo separato
            register(Name, Pid),  % Registra il nome del processo per facilitarne l'accesso
            {ok, Pid};  % Restituisce il Pid del processo creato
        _ ->
            {error, already_exists}
    end;
new(_, _, _, _) ->
    {error, invalid_parameters}.

% Funzione per creare un tab (una matrice NxM di celle)
create_tab(N, M) ->
    lists:map(fun(_) -> lists:duplicate(M, undef) end, lists:seq(1, N)).

% Loop che gestisce il processo del foglio di calcolo
loop(Name, Owner, N, M, K) ->
    Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
    Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
    loop(Spreadsheet).

% Loop che gestisce lo Stato
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies}) ->
    receive


        stop ->
            ok;  % Termina il processo
        _Other ->
            loop(State)  % Continua a ricevere messaggi
    end.



Il processo Spreadsheet è ora separato dal processo chiamante, e si possono inviare messaggi da shell o da altri processi.
Il processo Spreadsheet rimane in esecuzione in background con uno Stato persistente,  ricevendo messaggi fino a quando non viene fermato esplicitamente con stop.
Il processo chiamante che ha creato il foglio è salvato come proprietario, e posso implementare controlli di accesso basati su questo.

## funzione loop/1 aggiornata,  con aggiunta delle funzioni di lettura e scrittura

In [ ]:
% loop che gestisce lo State del foglio di calcolo e le operazioni sui dati
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies}) ->
    io:format("Spreadsheet State loop started. Waiting for messages.~n"),
    receive

        % Handle get request
        %la Tab(la matrice) restituita da lists:nth(Tab, Tabs) è assegnata a TabMatrix
        {get, From, Tab, I, J} ->
            io:format("Received get request for Tab: ~p, Row: ~p, Col: ~p~n", [Tab, I, J]),
            if
                Tab > length(Tabs) orelse Tab < 1 ->
                    io:format("Tab index ~p is out of bounds~n", [Tab]),
                    From ! {cell_value, undef};
                true ->
                    TabMatrix = lists:nth(Tab, Tabs),
                    if
                        I > length(TabMatrix) orelse I < 1 ->
                            io:format("Row index ~p is out of bounds in Tab ~p~n", [I, Tab]),
                            From ! {cell_value, undef}; %If the indices are out of bounds, we return undef instead of trying to access an invalid position.
                        true ->
                            Row = lists:nth(I, TabMatrix),
                            if
                                J > length(Row) orelse J < 1 ->
                                    io:format("Col index ~p is out of bounds in Row ~p, Tab ~p~n", [J, I, Tab]),
                                    From ! {cell_value, undef};
                                true ->
                                    Value = lists:nth(J, Row),

                                    From ! {cell_value, Value}
                            end
                    end
            end,
            loop(State);

% Handle set request
        {set, From, Tab, I, J, Val} ->
            io:format("Received set request for Tab: ~p, Row: ~p, Col: ~p, Val: ~p~n", [Tab, I, J, Val]),
            if
                Tab > length(Tabs) orelse Tab < 1 ->
                    io:format("Tab index ~p is out of bounds~n", [Tab]),
                    From ! {set_result, false};
                true ->
                    TabMatrix = lists:nth(Tab, Tabs),
                    if
                        I > length(TabMatrix) orelse I < 1 ->
                            io:format("Row index ~p is out of bounds in Tab ~p~n", [I, Tab]),
                            From ! {set_result, false};
                        true ->
                            Row = lists:nth(I, TabMatrix),
                            if
                                J > length(Row) orelse J < 1 ->
                                    io:format("Col index ~p is out of bounds in Row ~p, Tab ~p~n", [J, I, Tab]),
                                    From ! {set_result, false};
                                true ->
                                    io:format("Setting value ~p at Tab: ~p, Row: ~p, Col: ~p~n", [Val, Tab, I, J]),
                                    NewRow = replace_nth(J, Val, Row),
                                    NewTabMatrix = replace_nth(I, NewRow, TabMatrix),
                                    NewTabs = replace_nth(Tab, NewTabMatrix, Tabs),
                                    NewState = State#spreadsheet{tabs = NewTabs},
                                    From ! {set_result, true},
                                    loop(NewState)
                            end
                    end
            end;





        {get_spreadsheetPid, From} ->

                io:format("Received request for spreadsheet state from: ~p~n", [From]),
                io:format("State to be returned: ~p~n", [State]),  % Stampa il valore di `State`
                % Invia il record #spreadsheet{} indietro
                From ! {spreadsheet_state, State},
                io:format("Sent spreadsheet state to: ~p~n", [From]),
                loop(State);  % Continua ad ascoltare messaggi

        {share, From, NewPolicies} ->
            if
                From =:= Owner ->
                    % Aggiornare le politiche di accesso
                    NewState = State#spreadsheet{access_policies = NewPolicies},
                    From ! {share_result, ok},
                    loop(NewState);
                true ->
                    % Se non è il proprietario, ritorna un errore
                    From ! {share_result,{error, not_owner}},
                    loop(State)
            end;
        {remove_policy, From, Proc} ->
            if
                From =:= Owner ->

                    NewPolicies = lists:filter(fun({P, _}) -> P =/= Proc end, Policies),
                    NewState = State#spreadsheet{access_policies = NewPolicies},
                    From ! {remove_policy_result, ok},
                    loop(NewState);

                true ->
                    From ! {error, not_owner},
                    loop(State)
            end;
        % Gestire messaggi per operazioni sul foglio di calcolo
        stop ->

            ok; % esce dal loop , ma attenzione non arresta Spreadsheet !!!
        _Other ->
            io:format("Unknown message received: ~p~n", [_Other]),
            loop(State)
    end.

To do
info(name) -> Spreadsheet_info Le informazioni devono contenere almeno:
• Numerodi celle x tab
• I permessi di lettura e scrittura

testare le politiche di accesso e inserire il controllo di accesso su get e set
Controllare i valori ritornati dalle funzioni delle specifiche (bool, ok,  ..)



# Persistenza del foglio di calcolo sotto forma di CSV
Soddisfatti i requisiti iniziali con new/1,new/4,share/2 per la creazione e la gestione delle politiche di accesso il progetto deve soddisfare ulteriori requisiti:
1) garantire la persistenza del foglio di calcolo con salvataggio dei record #spreadsheet sotto forma  di file CSV tenendo presente che le funzioni di interfaccia richieste devono essere
to_cvs(filename, spreadsheet) - > ok | {error, reason}
from_cvs(filename) - > speadsheet | {error, reason}.
In più bisogna tener presente che il foglio di calcolo accetta valori di ogni tipo più undef

In [ ]:
% Funzione per salvare il foglio di calcolo in un file CSV, accetta il registered name  spreadsheet
to_csv(Filename, SpreadsheetName) ->
    % Recupera il PID associato al nome registrato
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            % Invia un messaggio al processo per ottenere lo stato del foglio di calcolo
            Pid ! {get_spreadsheet, self()},
            receive
                {spreadsheet_state, #spreadsheet{name = Name, tabs = Tabs}} ->
                    case file:open(Filename, [write]) of
                        {ok, File} ->
                            % Scrivi il nome del foglio
                            io:format(File, "Spreadsheet Name: ~s~n", [Name]),
                            % Salva ogni tab come riga CSV
                            lists:foreach(fun(Tab) -> save_tab_to_csv(File, Tab) end, Tabs),
                            file:close(File),
                            ok;
                        {error, Reason} ->
                            {error, Reason}
                    end
            after 5000 ->
                {error, timeout}
            end
    end.

% Funzione per salvare ogni tab come riga CSV
save_tab_to_csv(File, Tab) ->
    lists:foreach(fun(Row) ->
        Line = lists:map(fun(Cell) -> format_cell(Cell) end, Row),
        io:format(File, "~s~n", [string:join(Line, ",")])
    end, Tab).

% Formattazione delle celle per CSV
format_cell(undef) -> "undef";
format_cell(Cell) -> io_lib:format("~p", [Cell]).


#### Persistenza dell' intero record con le già viste to_csv/2 e from_csv/2

**to_csv/2**
```console
...
 case file:open(Filename, [write]) of
        {ok, File} ->
            % Write the spreadsheet name
            io:format(File, "Spreadsheet Name: ~s~n", [Name]),

            % Write the owner information
            io:format(File, "Owner: ~p~n", [Owner]),

            % Write the access policies
            io:format(File, "Access Policies: ~p~n", [AccessPolicies]),

            % Write each tab as a row of CSV
            lists:foreach(fun(Tab) -> save_tab_to_csv(File, Tab) end, Tabs),
            file:close(File),
            ok;
        {error, Reason} ->
            {error, Reason}
    end.
```
The owner (which is the PID of the process that owns the spreadsheet) is serialized using ~p, so it’s written in a way that can be properly restored later.
The access policies list is serialized as a complete list of tuples (e.g., {proc1, read}), which can be parsed back when loading.





## From_csv/1

Binding Variables Correctly: Each field (like SpreadsheetName, Owner, and AccessPolicies) is bound inside its respective case expression, so it’s used only within the correct scope. This avoids the unsafe variable error.

PID and Access Policies Conversion:

The owner field is converted back to a PID using list_to_pid/1.
The access policies are converted back to a valid Erlang term using list_to_term/1 (assuming the access policies were written as a valid term using ~p in the to_csv/2 function).
Tabs Loading: After reading the metadata (name, owner, and access policies), the function proceeds to load the tabs using load_tabs_from_csv(File).

Error Handling: If any part of the file format doesn’t match the expected pattern (e.g., missing "Owner:" or "Access Policies:"), the function will log an error and return {error, invalid_format}.

Converting PIDs and Terms:
list_to_pid/1: Converts a string representation of a PID (like "<0.88.0>") back into an Erlang PID.

The erl_scan:string/1 function is used to tokenize the string that represents the Erlang term.
The . is added to complete the term since the string representation does not include this.
Parsing the Tokens:

Once tokenized, erl_parse:parse_term/1 converts the tokens into a valid Erlang term.
This step handles the conversion from the string representation of {proc1, read} and {proc2, write} into proper Erlang terms.

In [ ]:
from_csv(Filename) ->
    case file:open(Filename, [read]) of
        {ok, File} ->
            % Read the first line (Spreadsheet Name)
            SpreadsheetNameLine = io:get_line(File, ''),
            io:format("Raw Spreadsheet Name Line: ~p~n", [SpreadsheetNameLine]),

            % Extract the spreadsheet name
            case string:strip(SpreadsheetNameLine, both, $\n) of
                "Spreadsheet Name: " ++ SpreadsheetName ->
                    io:format("Extracted Spreadsheet Name: ~p~n", [SpreadsheetName]),

                    % Read the owner field
                    OwnerLine = io:get_line(File, ''),
                    io:format("Raw Owner Line: ~p~n", [OwnerLine]),

                    % Extract the owner
                    case string:strip(OwnerLine, both, $\n) of
                        "Owner: " ++ Owner ->
                            io:format("Extracted Owner PID: ~p~n", [Owner]),

                            % Read the access policies field
                            AccessPoliciesLine = io:get_line(File, ''),
                            io:format("Raw Access Policies Line: ~p~n", [AccessPoliciesLine]),

                            % Extract the access policies
                            case string:strip(AccessPoliciesLine, both, $\n) of
                                "Access Policies: " ++ AccessPoliciesString ->
                                    io:format("Extracted Access Policies (String): ~p~n", [AccessPoliciesString]),

                                    % Convert access policies string to a valid term
                                    case parse_term_from_string(AccessPoliciesString) of
                                        {ok, AccessPolicies} ->
                                            io:format("Parsed Access Policies: ~p~n", [AccessPolicies]),

                                            % Load the tabs from the remaining lines in the CSV
                                            Tabs = load_tabs_from_csv(File),
                                            io:format("Read Tabs: ~p~n", [Tabs]),  % Debug

                                            % Close the file
                                            file:close(File),

                                            % Construct the spreadsheet record
                                            Spreadsheet = #spreadsheet{
                                                name = list_to_atom(SpreadsheetName),  % Convert the name to an atom
                                                tabs = Tabs,
                                                owner = list_to_pid(Owner),  % Convert owner to a PID
                                                access_policies = AccessPolicies  % Use the parsed term for access policies
                                            },

                                            {ok, Spreadsheet};
                                        {error, Reason} ->
                                            io:format("Error parsing Access Policies: ~p~n", [Reason]),
                                            file:close(File),
                                            {error, invalid_access_policies}
                                    end;

                                _ ->
                                    io:format("Error: Invalid format for Access Policies line~n"),
                                    file:close(File),
                                    {error, invalid_format}
                            end;

                        _ ->
                            io:format("Error: Invalid format for Owner line~n"),
                            file:close(File),
                            {error, invalid_format}
                    end;

                _ ->
                    io:format("Error: Invalid format for Spreadsheet Name line~n"),
                    file:close(File),
                    {error, invalid_format}
            end;

        {error, Reason} ->
            io:format("Error opening file: ~p~n", [Reason]),
            {error, Reason}
    end.

% Funzione ausialiara per interpretare una stringa come termine
parse_term_from_string(String) ->
    % Tokenize the string
    case erl_scan:string(String ++ ".") of  % Add a period to complete the expression
        {ok, Tokens, _} ->
            % Parse the tokens into an Erlang term
            case erl_parse:parse_term(Tokens) of
                {ok, Term} -> {ok, Term};
                {error, Reason} -> {error, Reason}
            end;
        {error, Reason, _} -> {error, Reason}
    end.


## Timeout per la receive
```css

after 5000 ->
    {error, timeout}
end

```

To implement the functions for reading and writing values in the spreadsheet cells (get/5 and set/6) and their versions with timeouts, we can modify the spreadsheet process to handle these operations.
get/5: Retrieve the value at position (i, j) in tab k of the spreadsheet.
set/6: Write a value to the position (i, j) in tab k of the spreadsheet.

Both operations should be able to:
Access the specific tab.
Index into the matrix representing the tab to read or write values.
Support a version with timeout (get/6 and set/7).
We will implement these functions by interacting with the spreadsheet process and handling messages like {get, From, Tab, I, J} and {set, From, Tab, I, J, Val} in the spreadsheet's loop.

In [ ]:
% La funzione get/5 spedisce un messaggio al processo spreadsheet richiedendo il valore di una specifica cella del foglio di calcolo
get(Name, Tab, I, J) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {get, self(), Tab, I, J},
    receive
        {cell_value, Value} -> Value
    after 5000 -> % Default timeout of 5 seconds
        timeout
    end.
% La funzione get/6 esegue lo stesso codice impostando un valore specifico per il timeout
get(Name, Tab, I, J, Timeout) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {get, self(), Tab, I, J},
    receive
        {cell_value, Value} -> Value
    after Timeout ->
        timeout
    end.


%La funzione set/6 spedisce un messaggio al processo spreadsheet per aggiornare una cella specifica con un nuovo valore
set(Name, Tab, I, J, Val) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {set, self(), Tab, I, J, Val},
    receive
        {set_result, Result} -> Result
    after 5000 -> % Default timeout of 5 seconds
        timeout
    end.

% set/7 specifica ulteriormente un valore di timeout desiderato
set(Name, Tab, I, J, Val, Timeout) ->
    Pid = whereis(Name),  % Get the PID of the spreadsheet process
    Pid ! {set, self(), Tab, I, J, Val},
    receive
        {set_result, Result} -> Result
    after Timeout ->
        timeout
    end.




# Aggiunta di specifiche di condivisione con politiche di accesso ai dati

share(spreadsheet, AccessPolicies) -> bool. Il proprietario del foglio può condidivere il foglio in Lettura o Scrittura con altri processi secondo AccessPolicies, che è una lista di {Proc,AP} dove:
 • Proc è un Pid/reg_name;
  • AP = read | write.
**Le policy di accesso ad un foglio possono cambiare in qualsiasi momento.**

Con la stessa share(spreadsheet, AccessPolicies) -> bool.
implemento  la gestione delle politiche di accesso (access policies) in modo da poter aggiornare solo le tuple {Proc, AP} presenti nella lista passata a share/2, mantenendo inalterate le altre

In breve:

Se un processo (Proc) è già presente nelle policy di accesso, il suo valore di accesso (AP) viene aggiornato con quello passato a share/2.
Se un processo (Proc) non è presente nella lista, viene aggiunto con il nuovo valore di accesso.
Le tuple {Proc, AP} non contenute nella lista passata a share/2 vengono mantenute inalterate.




In [ ]:

-module(spreadsheet).

%%.....stesso codice

% Funzione per condividere il foglio con policy di accesso
share(SpreadsheetName, AccessPolicies) when is_list(AccessPolicies) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {share, self(), AccessPolicies},  % Invia il nuovo messaggio per aggiornare le politiche
            receive
                {share_result, Result} -> Result
            end
    end.
    
% funzione ausiliarie di filtraggio per ottenere le Policy di Accesso aggiornate

    % List comprehension [Expression || Pattern <- List, Condition]
update_policies(NewPolicies, ExistingPolicies) ->
    % Step 1: Filter ExistingPolicies to exclude entries that are already in NewPolicies
    FilteredExistingPolicies = [
        Policy || 
        {Proc, _} = Policy <- ExistingPolicies, 
        not (
            lists:keymember(Proc, 1, NewPolicies) orelse
            is_pid(Proc) andalso lists:any(fun({NewProc, _}) -> whereis(NewProc) == Proc end, NewPolicies) orelse
            not is_pid(Proc) andalso lists:any(fun({NewProc, _}) -> NewProc == whereis(Proc) end, NewPolicies)
        )
    ],

    % Step 2: Return the combined list of NewPolicies and FilteredExistingPolicies
    FilteredExistingPolicies ++ NewPolicies.



% Loop principale del processo, dove si gestiscono i messaggi
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies}) ->
    receive
        {share, From, NewPolicies} ->
            if
                From =:= Owner ->
                    % Aggiornare le politiche di accesso usando update_policies/2
                    UpdatedPolicies = update_policies(NewPolicies, Policies),
                    NewState = State#spreadsheet{access_policies = UpdatedPolicies},
                    io:format("Updated access policies: ~p~n", [UpdatedPolicies]),  % Debug
                    From ! {share_result, true},
                    loop(NewState);
                true ->
                    % Se non è il proprietario, ritorna un errore
                    From ! {share_result, {error, not_owner}},
                    loop(State)
            end;
        stop ->
            ok; % Fermare il processo
        _Other ->
            loop(State)
    end.


I'm considering if we can merge in one list comprehension operation the act of not keeping in the merged list Newpolicies ++ Existing Policies  tuples that are in ExistingPolicies with {Proc, _} that is  lists:keymember(Proc, 1, NewPolicies) and tuples with {Proc, _} having Proc referring as a Pid o as a registered name to a member of Newpolicies with the first tuple"element referring to the same Process

In [ ]:
-module(upd_policies).
-export([update_policies/2]).
update_policies(NewPolicies, ExistingPolicies) ->
    % Step 1: Filter ExistingPolicies to exclude entries that are already in NewPolicies
    FilteredExistingPolicies = [
        Policy || 
        {Proc, _} = Policy <- ExistingPolicies, 
        not (
            lists:keymember(Proc, 1, NewPolicies) orelse
            is_pid(Proc) andalso lists:any(fun({NewProc, _}) -> whereis(NewProc) == Proc end, NewPolicies) orelse
            not is_pid(Proc) andalso lists:any(fun({NewProc, _}) -> NewProc == whereis(Proc) end, NewPolicies)
        )
    ],

    % Step 2: Return the combined list of NewPolicies and FilteredExistingPolicies
    FilteredExistingPolicies ++ NewPolicies.
    %With this single list comprehension, you can effectively:

    %Remove duplicates where the process in ExistingPolicies is already represented in NewPolicies (either as a PID or registered name).
    % Ensure that PIDs and registered names referring to the same process are handled correctly without introducing duplicates.


## implementazioni ulteriori sulla gestione delle politiche di accesso ai dati
2) Deve esserci un meccanismo per salvare le policy di accesso su disco e ricaricarle al riavvio del sistema (persistenza delle policy).
3) Una funzione deve permettere di rimuovere o modificare le policy di accesso esistenti


In [ ]:
% Funzione per rimuovere una policy di accesso per un processo specifico
remove_policy(SpreadsheetName, Proc) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {remove_policy, self(), Proc},
            receive
                {remove_policy_result, Result} -> Result
            end
    end.

% Funzione per salvare le politiche di accesso su disco
save_policies(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {save_policies, self()},
            receive
                {save_policies_result, Result} -> Result
            end
    end.

% Funzione per caricare le politiche di accesso da disco
load_policies(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {load_policies, self()},
            receive
                {load_policies_result, Result} -> Result
            end
    end.


## Le policy di accesso ai dati devono essere verificate nelle funzioni di lettura e scrittura

Per la lettura (get/4 e get/5), dobbiamo assicurarci che il processo chiamante abbia accesso in lettura.
Per la scrittura (set/5 e set/6), dobbiamo assicurarci che il processo chiamante abbia accesso in scrittura.
Introdurremo una funzione helper **check_access/3** per verificare se il processo chiamante ha i diritti di accesso corretti (lettura o scrittura). La funzione risolverà prima il processo chiamante e poi verificherà se il processo ha i diritti di accesso richiesti.

### Problemi affrontati
In un primo sviluppo check_access/3   only checks whether the PID (resolved from the registered name) is present in the access policies, but the original registered name (e.g., proc2) isn't being checked. As a result, if the registered name is in the policies but not the resolved PID, the function fails to find the matching policy, leading to {error, access_denied}. in un secondo sviluppo check_access/3 is currently checking the PID first, but the access policy only contains the registered name (proc2), not the PID (<0.98.0>). When the check_access/3 function is called with the PID, it doesn't find a matching policy for the PID in the list and returns {error, access_denied} even though the registered name has the appropriate access.


```console
check_access(PidOrName, Policies, RequiredAccess) ->
    % Resolve PidOrName to both PID and registered name if possible
    ResolvedPid = case is_pid(PidOrName) of
                      true -> PidOrName;  % If it's already a PID, keep it as the PID
                      false -> whereis(PidOrName)  % If it's a name, resolve to PID
                  end,

    % Find if the PID has a registered name
    RegisteredName = case is_pid(PidOrName) of
                         true -> find_registered_name(ResolvedPid);  % Get registered name for the PID
                         false -> PidOrName  % If it's already a name, keep it
                     end,

    % Check both the resolved PID and the registered name in the access policies
    case lists:keyfind(ResolvedPid, 1, Policies) of
        {ResolvedPid, Access} when Access == RequiredAccess -> ok;
        _ ->
            case lists:keyfind(RegisteredName, 1, Policies) of
                {RegisteredName, Access} when Access == RequiredAccess -> ok;
                _ -> {error, access_denied}
            end
    end.

% Helper function to find the registered name of a process by its PID
find_registered_name(Pid) ->
    lists:foldl(
        fun(Name, Acc) ->
            case whereis(Name) of
                Pid when Pid =/= undefined -> Name;  % Return the name if it matches the PID
                _ -> Acc  % Otherwise, keep searching
            end
        end,
        undefined,
        registered()
    ).

```


% Modifiche alle funzioni get/4 e get/5
```console
get(SpreadsheetName, TabIndex, I, J) ->
    get(SpreadsheetName, TabIndex, I, J, infinity). %infiniti riconduce get/4 a get/5

get(SpreadsheetName, TabIndex, I, J, Timeout) ->
    case whereis(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};
        Pid ->
            % Check if the caller has read access
            %Prima di inviare la richiesta get al processo del foglio di calcolo, controlliamo se il processo %chiamante (self()) ha accesso in lettura. In caso contrario, restituiamo {error, access_denied}.
            case check_access(self(), Spreadsheet#spreadsheet.access_policies, read) of
                ok ->
                    Pid ! {get, self(), TabIndex, I, J},
                    receive
                        {get_result, Value} -> Value
                    after Timeout -> {error, timeout}
                    end;
                {error, access_denied} -> {error, access_denied}
            end
    end.
```

% Modifiche alle funzioni set/5 e set/6
```console
set(SpreadsheetName, TabIndex, I, J, Value) ->
    set(SpreadsheetName, TabIndex, I, J, Value, infinity).

set(SpreadsheetName, TabIndex, I, J, Value, Timeout) ->
    case whereis(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};
        Pid ->
            % Check if the caller has write access
            %Prima di inviare la richiesta di set al processo del foglio di calcolo, controlliamo se il processo %chiamante (self()) ha accesso in scrittura. In caso contrario, restituiamo {error, access_denied}.
            case check_access(self(), Spreadsheet#spreadsheet.access_policies, write) of
                ok ->
                    Pid ! {set, self(), TabIndex, I, J, Value},
                    receive
                        {set_result, Result} -> Result
                    after Timeout -> {error, timeout}
                    end;
                {error, access_denied} -> {error, access_denied}
            end
    end.

```

## Aggiunta di funzioni per la persistenza delle policy di accesso

In [ ]:
-module(spreadsheet).
-export([new/1, new/4, share/2, remove_policy/2, save_policies/1, load_policies/1]).

%%.............

% Funzione per condividere il foglio con policy di accesso
share(SpreadsheetName, AccessPolicies) when is_list(AccessPolicies) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {share, self(), AccessPolicies},
            receive
                {share_result, Result} -> Result
            end
    end.

% Funzione per rimuovere una policy di accesso per un processo specifico
remove_policy(SpreadsheetName, Proc) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {remove_policy, self(), Proc},
            receive
                {remove_policy_result, Result} -> Result
            end
    end.
% Funzione per salvare le politiche di accesso su disco
{save_policies, From} ->
            % Codice per salvare le politiche su disco (implementazione seguente)
            % Serializzare le policy e salvarle in un file
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [write]),
            ok = io:format(File, "~p.", [Policies]),
            file:close(File),
            From ! {save_policies_result, ok},
            loop(State);


% Funzione per caricare le politiche di accesso da disco
load_policies(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid ->
            Pid ! {load_policies, self()},
            receive
                {load_policies_result, Result} -> Result
            end
    end.

% Loop che gestisce il processo del foglio di calcolo
loop(Name, Owner, N, M, K) ->
    Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
    Spreadsheet = #spreadsheet{name = Name, tabs = Tabs, owner = Owner},
    loop(Spreadsheet).

% Loop
loop(State = #spreadsheet{owner = Owner, access_policies = Policies}) ->
    receive
        {share, From, AccessPolicies} ->
            if
                From =:= Owner ->
                    NewState = State#spreadsheet{access_policies = AccessPolicies},
                    From ! {share_result, ok},
                    loop(NewState);
                true ->
                    From ! {share_result, {error, not_owner}},
                    loop(State)
            end;

        {remove_policy, From, Proc} ->
            if
                From =:= Owner ->
                    NewPolicies = lists:filter(fun({P, _}) -> P =/= Proc end, Policies),
                    NewState = State#spreadsheet{access_policies = NewPolicies},
                    From ! {remove_policy_result, ok},
                    loop(NewState);
                true ->
                    From ! {remove_policy_result, {error, not_owner}},
                    loop(State)
            end;





        {save_policies, From} ->
            % Codice per salvare le politiche su disco (implementazione seguente)
            % Serializzare le policy e salvarle in un file
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [write]),
            ok = io:format(File, "~p.", [Policies]),
            file:close(File),
            From ! {save_policies_result, ok},
            loop(State);

        {load_policies, From} ->
            % Codice per caricare le politiche da disco (implementazione seguente)
            {ok, File} = file:open(State#spreadsheet.name ++ "_policies.dat", [read]),
            {ok, Policies} = io:fread(File, "~p"),
            file:close(File),
            NewState = State#spreadsheet{access_policies = Policies},
            From ! {load_policies_result, ok},
            loop(NewState);

        stop -> ok;
        _Other -> loop(State)
    end.




# Gestione del possesso della proprietà del processo spreadsheet
To handle the situation where the Erlang shell crashes and you lose ownership of the spreadsheet process (because the owner is tied to the original shell's PID), you can implement a mechanism that allows the owner to be reassigned to a new process or PID. This is especially important in a fault-tolerant system where processes can crash, and we need to recover gracefully.


Allow Reassigning Ownership: 
Implement a new function (e.g., reassign_owner/2) that allows changing the #spreadsheet.owner field to a new PID (the current shell's PID).
Ownership Check: Ensure that only the current owner or an administrator/special role can trigger this reassignment.
Error Handling: Handle errors gracefully if an unauthorized process tries to reassign ownership.


In [ ]:
% Funzione che permette di riassegnare  il proprietario  di spreadsheet se il processo chiamamante è il corrente proprietario
reassign_owner(SpreadsheetName, NewOwnerPid) when is_pid(NewOwnerPid) ->
    case whereis(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found}; % The spreadsheet process doesn't exist
        Pid ->
            Pid ! {reassign_owner, self(), NewOwnerPid},
            receive
                {reassign_owner_result, Result} -> Result
            end
    end.

% modifiche a loop/1
loop(State = #spreadsheet{name = Name, owner = Owner} = SpreadsheetState) ->
    receive
        % Handle ownership reassignment
        {reassign_owner, From, NewOwner} ->
            if
                From =:= Owner ->  % Only the current owner can reassign ownership
                    NewState = State#spreadsheet{owner = NewOwner},
                    io:format("Ownership reassigned from ~p to ~p~n", [Owner, NewOwner]),
                    From ! {reassign_owner_result, ok},
                    loop(NewState);
                true ->
                    From ! {reassign_owner_result, {error, unauthorized}},
                    loop(State)
            end;

        % Existing handlers...

        % Stop the spreadsheet process
        stop ->
            ok;
            
        _Other ->
            io:format("Unknown message received: ~p~n", [_Other]),
            loop(State)
    end.


# implementare la funzione info/1, che fornirà informazioni dettagliate sul foglio di calcolo, tra cui:

**Numero di celle per scheda** (lo calcoleremo contando il numero di righe e colonne in ogni scheda e moltiplicando i valori).
**Autorizzazioni di lettura e scrittura**: forniremo una panoramica dei processi che hanno accesso in lettura e scrittura in base al campo access_policies.

**Numero totale di schede**: quante schede (fogli di lavoro) ha il foglio di calcolo.
**Proprietario del foglio di calcolo**: visualizza il proprietario del foglio di calcolo.
**Numero totale di celle**: somma il numero totale di celle in tutte le schede.
Includi la **data dell'ultima modifica** dei dati del foglio di calcolo

La funzione prende il nome (o PID) del foglio di calcolo come input.
Interroga il processo del foglio di calcolo per conoscere il suo stato (tramite un messaggio), da cui estrarrà le informazioni richieste e le restituirà in modo strutturato.

In [ ]:
-record(spreadsheet, {
    name,
    tabs = [],                 % List of tabs (each tab is a matrix NxM)
    owner = undefined,          % Owner of the spreadsheet
    access_policies = [],       % List of {Proc, Access} tuples
    last_modified = undefined   % Timestamp of the last modification
}).

% Function to get spreadsheet information
info(SpreadsheetName) ->
    case whereis(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};  % If the process is not found
        Pid ->
            % Send a request to the spreadsheet process to get its state
            Pid ! {get_info, self()},
            receive
                {info_result, Info} -> {ok, Info};
                {error, Reason} -> {error, Reason}
            after 5000 -> {error, timeout}
            end
    end.
loop(State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = AccessPolicies}) ->
    receive
        {get_info, From} ->
            % Calculate the number of cells and total tabs
            TotalTabs = length(Tabs),
            TotalCells = lists:sum([length(Tab) * length(lists:nth(1, Tab)) || Tab <- Tabs]),
            
            % Split access policies into read and write permissions
            ReadPermissions = [Proc || {Proc, read} <- AccessPolicies],
            WritePermissions = [Proc || {Proc, write} <- AccessPolicies],

            % Create the info result
            Info = #{name => Name,
                     owner => Owner,
                     total_tabs => TotalTabs,
                     total_cells => TotalCells,
                     read_permissions => ReadPermissions,
                     write_permissions => WritePermissions},

            % Send the result back to the requester
            From ! {info_result, Info},
            loop(State);

        % Other cases...

        

        _Other ->
            io:format("Unknown message received: ~p~n", [_Other]),
            loop(State)
    end.


In [ ]:
% set/5 function (with infinite timeout)
set(SpreadsheetName, TabIndex, I, J, Value) ->
    set(SpreadsheetName, TabIndex, I, J, Value, infinity).

% set/6 function (with custom timeout)
set(SpreadsheetName, TabIndex, I, J, Value, Timeout) ->
    case whereis(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};
        Pid ->
            % Send the set request to the spreadsheet process
            Pid ! {set, self(), TabIndex, I, J, Value},
            receive
                {set_result, ok} -> ok;
                {error, access_denied} -> {error, access_denied}
            after Timeout ->
                {error, timeout}
            end
    end.


In [ ]:
to_csv e from_csv ...


## Here comes the approach to distributed version

# Distributed Erlang Basics

In a distributed Erlang system, multiple Erlang nodes communicate with each other over a network. Each node is identified by a unique name (e.g., node1@hostname, node2@hostname), and nodes can be connected to allow processes on different nodes to communicate.

Setting Up Distributed Erlang:
You can start a distributed Erlang node using the -sname or -name option:


`erl -sname node1  % Short name (local domain resolution)`
`erl -name node1@hostname  % Full name (DNS resolution)`
Nodes can then connect using net_adm:ping(Node):


`net_adm:ping('node2@hostname').`
You can check connected nodes with:


`nodes().`


## Global Process Registration in a Distributed System

Global process registration in distributed Erlang ensures that a process registered on one node can be located and accessed from any other connected node.

Using Global Registration with Distributed Nodes:
With distributed nodes, the global process registry will synchronize process registrations across nodes, allowing you to register a process under a unique name that can be resolved by any node.

For example, on node1@hostname:


`gen_server:start_link({global, my_server}, ?MODULE, Args, []).`

And on node2@hostname, you can look up the process by name:


`global:whereis_name(my_server).`

Considerations with Global Registration:
Global Name Conflicts: Since process names must be unique across nodes, you may run into name conflicts if two nodes try to register processes under the same name.
Synchronization Delays: There can be slight delays in name synchronization between nodes, which could lead to noproc issues if one node hasn’t fully updated its registry.
Node Failures: If a node crashes, the global registry will eventually remove processes registered on that node, but there may be a lag during the failure recovery.



### Choosing Between Global and Local Registration in Distributed Erlang

In a distributed system, you can choose to either use global registration or a combination of local registration and node awareness. Here’s when to use each approach:

When to Use Global Registration:
Process Lookup Across Nodes: If processes on different nodes need to discover each other dynamically by name.

Fault Tolerance Across Nodes: If you want to enable failover mechanisms across nodes, where processes on one node can recover or take over the responsibilities of processes on another node.

For example, registering a critical process like a master controller globally ensures that any node can resolve the master’s process regardless of where it’s running.

When to Use Local Registration (with Node Awareness):
Process Lookup Within a Single Node: If the process will only ever be looked up or accessed on the local node.
Node-Specific Responsibilities: If each node has distinct responsibilities, and processes are node-specific, local registration reduces complexity and overhead.
In this approach, processes can communicate directly between nodes using PIDs or node names, without relying on the global registry.

## Handling Global Registration Across Nodes
Step 1: Start Distributed Erlang Nodes
To start Erlang in distributed mode, ensure that the nodes are properly named and reachable. For example:


`erl -sname node1`
`erl -sname node2`

Step 2: Connect the Nodes
In the Erlang shell, you can connect the nodes using:


`net_adm:ping('node2@hostname').`
This establishes communication between node1 and node2.

Step 3: Register Processes Globally
Once the nodes are connected, you can register processes globally on any node, and they will be accessible across the cluster. For example, start a gen_server on node1 and register it globally:


`gen_server:start_link({global, my_spreadsheet_server}, ?MODULE, Args, []).`
On another node (e.g., node2), you can now resolve the process using:


`global:whereis_name(my_spreadsheet).`

### Applying gen_server to a Distributed Spreadsheet
When designing a distributed spreadsheet with Erlang, using gen_server is indeed appropriate because it provides robust process management and simplifies handling of state and communication. The gen_server framework can efficiently manage the complexities associated with a distributed environment, such as concurrency, fault tolerance, and state consistency.
Key Considerations for a Distributed Spreadsheet:
**State Management**: The spreadsheet can be represented as a record in Erlang, which includes not just the data but also metadata such as access policies and possibly a revision history to manage changes over time. The state of the gen_server would encapsulate this record.
**Concurrency and Access Control**: Handling concurrent access to different parts of the spreadsheet requires careful management of state to ensure data integrity and consistency. gen_server's sequential handling of requests can naturally serialize access to data, but for finer-grained concurrency control (e.g., allowing simultaneous edits in different tabs), additional mechanisms might be needed.
Distributed Operations: Since the spreadsheet is distributed, operations may need to span multiple nodes. The gen_server can handle local operations, and for operations that need consistency across nodes, you might use distributed Erlang features or a distributed database like Mnesia.
**Node Failure and Recovery**: One of the strengths of using gen_server within the OTP framework is its built-in support for handling failures through supervisors. You can design your system to automatically restart failed spreadsheet processes, possibly restoring their last known good state from a backup or log.

### Overview of **gen_server**
gen_server is a generic server behavior that abstracts the common functionalities of a server process in Erlang/OTP. It provides a framework for implementing servers, handling everything from message passing and event handling to state management and concurrency. By using gen_server, *developers can focus on the business logic specific to their application while relying on the proven, robust infrastructure provided by OTP for handling process life cycles, communication, and error recovery.*
Components of gen_server
gen_server consists of several key components:

Callbacks: You implement specific callback functions that gen_server will call during its operation. These include:

init/1: Initializes the server. This is called when the server starts.
handle_call/3: Handles synchronous calls to the server.
handle_cast/2: Handles asynchronous messages to the server.
handle_info/2: Handles all other messages that don't fit into the above categories.
terminate/2: Cleans up when the server is shutting down.
code_change/3: Handles dynamic code upgrades.
API Functions: These are functions you define to interact with the server. They usually wrap gen_server calls like gen_server:call() and gen_server:cast().






In gen_server, you use handle_cast/2 instead of handle_call/3 when you want to handle asynchronous messages. The key difference between these two callbacks is whether the message being handled expects a reply or not:

handle_call/3:

Used for synchronous requests.
The caller waits for a reply.
Example: gen_server:call(Server, Request).
handle_cast/2:

Used for asynchronous messages.
The caller does not wait for a reply, and the server doesn't send one.
Example: gen_server:cast(Server, Request)

## Setup GenServer Skeleton


-module(spreadsheet).
-behaviour(gen_server).

%% API
-export([start_link/1, stop/0, create_tab/2, remove_policy/2, update_policies/2]).

%% gen_server callbacks
-export([init/1, handle_call/3, handle_cast/2, handle_info/2, terminate/2, code_change/3]).

%% Include the record definition
-include("spreadsheet.hrl").

%%% API FUNCTIONS %%%

%% Start the spreadsheet process with a given name
start_link(Name) ->
    gen_server:start_link({global, Name}, ?MODULE, Name, []).

%% Stop the spreadsheet process
stop() ->
    gen_server:cast(self(), stop).

%% Create a new tab (NxM)
create_tab(SpreadsheetName, {N, M}) ->
    gen_server:call(SpreadsheetName, {create_tab, N, M}).

%% Remove an access policy for a specific process
remove_policy(SpreadsheetName, Proc) ->
    gen_server:call(SpreadsheetName, {remove_policy, Proc}).

%% Update access policies
update_policies(SpreadsheetName, NewPolicies) ->
    gen_server:cast(SpreadsheetName, {update_policies, NewPolicies}).

%%% CALLBACK FUNCTIONS %%%

%% Initialize the spreadsheet process
init(Name) ->
    State = #spreadsheet{name = Name, tabs = [], owner = undefined, access_policies = [], last_modified = undefined},
    {ok, State}.

%% Handle synchronous calls (like create_tab and remove_policy)
handle_call({create_tab, N, M}, _From, State) ->
    NewTab = create_tab(N, M),
    NewState = State#spreadsheet{tabs = [NewTab | State#spreadsheet.tabs]},
    {reply, ok, NewState};

handle_call({remove_policy, Proc}, _From, State) ->
    %% Original remove_policy logic
    NewPolicies = lists:filter(fun({P, _}) -> P =/= Proc end, State#spreadsheet.access_policies),
    NewState = State#spreadsheet{access_policies = NewPolicies},
    {reply, ok, NewState};

handle_call(_Request, _From, State) ->
    {reply, {error, unknown_request}, State}.

%% Handle asynchronous casts (like update_policies)
handle_cast({update_policies, NewPolicies}, State) ->
    %% Original update_policies logic
    FilteredExistingPolicies = lists:filter(
        fun({Proc, _}) ->
            not lists:keymember(Proc, 1, NewPolicies)
        end, State#spreadsheet.access_policies),
    NewState = State#spreadsheet{access_policies = FilteredExistingPolicies ++ NewPolicies},
    {noreply, NewState};

handle_cast(stop, State) ->
    {stop, normal, State}.

%% Handle any other messages
handle_info(_Info, State) ->
    {noreply, State}.

%% Clean up when the process stops
terminate(_Reason, _State) ->
    ok.

%% Handle code change if the module is upgraded
code_change(_OldVsn, State, _Extra) ->
    {ok, State}.

%%% HELPER FUNCTIONS %%%

%% Create a new tab with NxM cells
create_tab(N, M) ->
    lists:map(fun(_) -> lists:duplicate(M, undef) end, lists:seq(1, N)).


## Handling State
The spreadsheet record (#spreadsheet) holds the entire state. The key part of the gen_server state consists of:

tabs: The spreadsheet data (list of 2D matrices).
owner: The process that owns the spreadsheet.
access_policies: Policies related to access.
last_modified: A timestamp to track changes.
We manage state updates inside the respective handlers (handle_call/3 and handle_cast/2).

## Distributed Operation
Since we use gen_server:start_link({global, Name}, ...), the spreadsheet processes are globally registered by name. This means that spreadsheets can be distributed across nodes in a cluster.

Starting the Spreadsheet Server: When we call start_link(Name), it registers the spreadsheet under a globally unique name, allowing it to be accessed from any node in the distributed system.
Inter-Node Communication: All subsequent API calls (create_spreadsheet/2, update_cell/4, get_cell/3, etc.) interact with the registered global process by name.

## Key Concepts to Refactor:
new/1 and new/4:
These functions currently spawn the spreadsheet process and pass the parameters required to initialize the state (using starter/6).
starter/6:
Initializes the state by creating a list of tabs (2D matrices), setting the owner, and defining initial access policies.
Ends with a loop/1 function to manage the spreadsheet’s state in a receive loop (process-level state management).
We want to maintain your existing API calls (new/1, new/4) while refactoring the process spawning and loop management into a gen_server.
The loop/1 in your current implementation manages the state, but in a gen_server, state is managed within the callback functions (handle_call/3, handle_cast/2, etc.).
Instead of explicitly spawning a process with spawn/3, we will now use gen_server:start_link/3 to start the process and manage the state within the gen_server.

gen_server:start_link({global, Name}, ...) automatically registers the process globally under Name, much like using global:register_name(Name, Pid).
This simplifies the process registration, as you don’t need to manually register the process after starting it.
The process will be globally visible across nodes in a distributed Erlang environment, allowing for easy lookup using global:whereis_name/1.

Ai fini di sviluppo e test viene creato un nuovo file per il modulo -module(distributed_spreadsheet)

In [ ]:
-module(distributed_spreadsheet). %ver 1.2 
-behaviour(gen_server).

%% API functions exported
-export([new/1, new/4, reassign_owner/2,
         info/1, stop/0]). that's the module with  

%% gen_server callbacks
-export([init/1, handle_call/3, handle_cast/2, handle_info/2, terminate/2, code_change/3]).

%% Include the record definition
-include("spreadsheet.hrl").

%%% API FUNCTIONS %%%


% Funzione per creare un nuovo foglio di nome Name e dimensioni definite internamente
new(Name) ->
    N = 5,  % Default N (numero di righe)
    M = 5,  % Default M (numero di colonne)
    K = 2,   % Default K (numero di tab)
    new(Name, N, M, K).
    

%% Funzione per creare un nuovo foglio con dimensioni  passate per valori
new(Name, N, M, K) when is_integer(N), is_integer(M), is_integer(K), N > 0, M > 0, K > 0 ->    % guardie sui valori passati->
    %gen_server:call(Name, {new, Name, N, M, K}).
    case global:whereis_name(Name) of
        undefined ->
            LastModified= erlang:system_time(),%calendar:universal_time(),
            Owner =self(), %Il Pid del processo chiamante viene salvato in Owner 
            %Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
        
            gen_server:start_link({global, Name}, ?MODULE, {Name, Owner, N, M, K, LastModified}, []);
            % Restituisce il Pid del processo creato, per inviargli messaggi                       
        _ ->
            {error, already_exists}
    end;
new(_, _, _, _) ->
    {error, invalid_parameters}.
%% Reassign the owner of the spreadsheet
reassign_owner(SpreadsheetName, NewOwner) ->
    gen_server:cast(SpreadsheetName, {reassign_owner, NewOwner}).

%% Stop the spreadsheet process
stop() ->
    gen_server:cast(self(), stop).



%% Get information about the spreadsheet
info(SpreadsheetName) ->
     case global:whereis_name(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};  % If the process is not found
        _Pid ->
            %% Use gen_server:call/2 to request the spreadsheet's state information
            gen_server:call(SpreadsheetName, get_info)
    end.

%%% gen_server CALLBACKS %%%

%% Initialize the spreadsheet process

init({Name, Owner, N, M, K, LastModified}) ->
    %% Init with  parameters from new/4
    io:format("~p process .~n", [Name]),
    Tabs = lists:map(fun(_) -> create_tab(N, M) end, lists:seq(1, K)),
    Policies = [{Owner, write}],  % Initial access policy
    State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies, last_modified = LastModified},
    {ok, State}.

%% Handle synchronous calls(e.g., getting a cell, or getting spreadsheet info)
%% Handle the synchronous request to get the spreadsheet's info
handle_call(get_info, _From,  State=#spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies, last_modified = LastModified}) ->
    io:format("want info from ~p~n", [_From]),
    TotalTabs = length(Tabs),
    TotalCells = lists:sum([length(Tab) * length(lists:nth(1, Tab)) || Tab <- Tabs]),
    
 %% Split access policies into read and write permissions
    ReadPermissions = [Proc || {Proc, read} <- Policies],
    WritePermissions = [Proc || {Proc, write} <- Policies],

    %% Create the info result map
    Info = #{name => Name,
             owner => Owner,
             last_modified => LastModified,
             total_tabs => TotalTabs,
             total_cells => TotalCells,
             read_permissions => ReadPermissions,
             write_permissions => WritePermissions
            },
    {reply, {ok, Info}, State};

handle_call({get, TabIdx, Row, Col}, _From, State) ->
    Tabs = State#spreadsheet.tabs,
    CellValue = lists:nth(Col, lists:nth(Row, lists:nth(TabIdx, Tabs))),
    {reply, {ok, CellValue}, State}.




%% Handle asynchronous casts (e.g., updating a cell, sharing, reassigning ownership)


handle_cast(stop, State) ->
    {stop, normal, State}.

%% Handle any other messages
handle_info(_Info, State) ->
    {noreply, State}.

%% Clean up when the process stops
terminate(_Reason, _State) ->
    ok.

%% Handle code change if the module is upgraded
code_change(_OldVsn, State, _Extra) ->
    {ok, State}.

%%% HELPER FUNCTIONS %%%

%% Create a new tab with NxM cells
create_tab(N, M) ->
    lists:map(fun(_) -> lists:duplicate(M, undef) end, lists:seq(1, N)).

## Overview of the gen_server Callbacks:
handle_call/3: Used for synchronous requests, where the calling process waits for a reply from the gen_server before proceeding. It ensures the calling process knows when the request has been handled.
handle_cast/2: Used for asynchronous requests, where the calling process sends a message to the gen_server and does not wait for a response. The process continues execution without knowing if or when the request was handled.

Argument 1: Ensuring Confirmation of Operation Completion
When using handle_call/3, the calling process gets a response (either success or failure) after the operation completes. This is crucial in situations where you need to ensure that the action (e.g., updating a cell, reassigning ownership) has been successfully carried out before the calling process can proceed.

Examples:

Updating a Cell (set/5):
The caller should know if the cell was updated successfully or if there was an error (e.g., the cell was out of bounds). Using handle_call/3, the caller waits for confirmation that the value has been set.
If you used handle_cast/2, the calling process wouldn’t know if the update succeeded or failed, potentially leading to inconsistencies.
Reassigning Ownership (reassign_owner/2):
When reassigning ownership, it’s important to confirm whether the ownership was reassigned successfully (and whether the caller was authorized to make that change). By using handle_call/3, the caller gets a response indicating whether the operation succeeded or failed.
With handle_cast/2, the calling process would have no way of knowing if the operation completed or encountered an issue, which is undesirable for such a critical operation.
Argument 2: Error Handling and Validation
Using handle_call/3 gives you a clean and structured way to handle errors and invalid operations. The gen_server can:

Perform validation (e.g., checking permissions, bounds for accessing or setting cells).
Return an appropriate response (e.g., {error, access_denied} or {ok, Value}).
Examples:

Sharing Access (share/2):
The caller needs to know whether the requested access policies were successfully applied or if there were any issues (e.g., invalid process, invalid policy). Using handle_call/3, the caller gets feedback, and you can easily propagate any validation errors (e.g., invalid access policy) back to the caller.
With handle_cast/2, you lose the ability to immediately inform the caller about such errors.
Fetching Spreadsheet Information (info/1):
When requesting spreadsheet metadata (e.g., total tabs, access permissions), the calling process needs to wait until the information is retrieved and returned, which makes handle_call/3 ideal.
If this were done with handle_cast/2, the calling process would not receive the response directly, making the operation less useful.
Argument 3: Consistency and Control Over State
Using handle_call/3 ensures that operations are carried out sequentially and consistently. The gen_server processes each request one at a time, ensuring that any changes to the state (e.g., updating a cell, adding access policies) are performed in a controlled manner.

State Mutations: Operations like modifying the spreadsheet, reassigning ownership, or setting cell values involve changing the internal state of the gen_server. With handle_call/3, the server ensures that each state modification is completed before handling the next request, avoiding race conditions or inconsistencies.
Examples:

Setting a Cell (set/5):
When a cell is updated, you want to ensure that no two requests are modifying the same part of the state concurrently. By using handle_call/3, the server processes each request in order, ensuring consistency.
With handle_cast/2, state changes are asynchronous, so there could be cases where requests overlap, potentially leading to race conditions if the server's state is modified simultaneously by different operations.
Argument 4: Better Debugging and Monitoring
Using handle_call/3 allows for easier debugging and monitoring of requests, since:

The caller waits for a response, so you can trace how long operations take and whether they succeed or fail.
You can use logging within the handle_call/3 callback to track the flow of requests and the responses sent back to the caller.
This makes it easier to understand how requests are processed and troubleshoot issues, as you can trace the lifecycle of each request and its response.



# MUOVI IN BASSO

In [ ]:
## Refactor info/1

info(SpreadsheetName) ->
    case global:whereis_name(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};  % If the process is not found
        Pid ->
            %% Use gen_server:call/2 to request the spreadsheet's state information
            gen_server:call(SpreadsheetName, get_info)
    end.
...
%% Handle the synchronous request to get the spreadsheet's info
handle_call(get_info, _From, State = #spreadsheet{name = Name, tabs = Tabs, owner = Owner, access_policies = Policies, last_modified = LastModified}) ->
    %% Log the request for info
    io:format("want info from ~p~n", [_From]),

    %% Calculate the total number of tabs and cells
    TotalTabs = length(Tabs),
    TotalCells = lists:sum([length(Tab) * length(lists:nth(1, Tab)) || Tab <- Tabs]),

    %% Split access policies into read and write permissions
    ReadPermissions = [Proc || {Proc, read} <- Policies],
    WritePermissions = [Proc || {Proc, write} <- Policies],

    %% Create the info result map
    Info = #{name => Name,
             owner => Owner,
             last_modified => LastModified,
             total_tabs => TotalTabs,
             total_cells => TotalCells,
             read_permissions => ReadPermissions,
             write_permissions => WritePermissions
            },

    %% Reply with the info result
    {reply, {ok, Info}, State}.




Rewriting info/1 for Debugging
info(SpreadsheetName) ->
    %% Step 1: Check if the process is registered globally
    case global:whereis_name(SpreadsheetName) of
        undefined -> 
            io:format("Spreadsheet ~p not found globally.~n", [SpreadsheetName]),
            {error, spreadsheet_not_found};
        
        Pid ->
            %% Step 2: Check if the process is alive
            io:format("Spreadsheet ~p is registered globally with PID ~p~n", [SpreadsheetName, Pid]),
            
            case erlang:is_process_alive(Pid) of
                true ->
                    %% Step 3: Optionally check the system status of the process
                    Status = sys:get_status(Pid),
                    io:format("Spreadsheet process status: ~p~n", [Status]),

                    %% Step 4: Try to make the gen_server:call and handle potential failures
                    try
                        %gen_server:call(SpreadsheetName, get_info)
                         gen_server:call(Pid, get_info)
                    catch
                        Class:Reason ->
                            io:format("Error calling gen_server:call/2: ~p, ~p~n", [Class, Reason]),
                            {error, {call_failed, Reason}}
                    end;
                
                false ->
                    io:format("Process ~p is not alive.~n", [Pid]),
                    {error, process_not_alive}
            end
    end.

Key Debugging Steps:
Process Registration: Check if global:whereis_name(SpreadsheetName) correctly returns a PID.
Check if the Process is Alive: The **erlang:is_process_alive(Pid)** function is now used inside a case statement to check if the process is alive.
Process Status: Use sys:get_status/1 to get the detailed status of the process.
Error Logging: Log any errors that occur during gen_server:call/2.


Why You Can't Call gen_server:call(SpreadsheetName, get_info) but Can Call gen_server:call(Pid, get_info)?
The issue where gen_server:call(SpreadsheetName, get_info) fails, but gen_server:call(Pid, get_info) works, could be related to name registration or distribution in Erlang
The From parameter in handle_call/3 is the identity of the caller process, and it is used internally by gen_server to send a reply back to the caller. The structure {CallerPid, Alias} indicates that the caller process might be using an alias or a reference when it sends the call.


# Summary of Differences Between Global and Local Registration:
(table)
Feature	Global Registration	Local Registration
Scope	Across all nodes in a distributed system	Limited to the current node
Name Resolution	Via global:whereis_name/1	Via whereis/1 (local)
Performance	Higher overhead due to global synchronization	Faster name resolution (local)
Complexity	Can lead to name conflicts and aliasing issues	Simpler, no aliasing issues
Use Case	Needed in distributed environments	Ideal for single-node systems
Error-prone	More prone to noproc issues due to global delays	Less error-prone, direct access to process

to move to hints
Tracer
Start the tracer using dbg:start().
Set the tracer with dbg:tracer().
Enable tracing for all processes with dbg:p(all, call).
Set a trace pattern for distributed_spreadsheet:info/1 using dbg:tpl(Module, Function, Arity, []).
Call the function and observe the trace output.
Stop the tracer using dbg:stop_clear().
Pattern Matching Failure
If the State passed to handle_call/3 does not match the expected structure of the #spreadsheet record, pattern matching will fail, and the process will crash.
Summary of Problems and Solutions(table):
Issue	Cause	Solution
Pattern Matching Failure	If the State is not a #spreadsheet record or does not have the expected fields.	Use defensive pattern matching (is_record/2) or handle unexpected states gracefully.
Performance (Minor)	Matching large records with many fields can add slight overhead.	Use partial matching or access specific fields via State#recordname.fieldname.
Record Field Defaults	Fields with default values may cause crashes if uninitialized.	Ensure all fields are initialized when creating the record.
Future Record Changes	Adding new fields to the record in the future may cause crashes in existing pattern matches.	Match only the fields you need, or use more flexible access to fields.


# Refactor share/2
Instead of using Pid ! {share, self(), AccessPolicies} and manually receiving the response, we'll use gen_server:call/2 to handle the synchronous interaction. This allows for cleaner and safer communication.


In [ ]:
%% API function to share access policies
share(SpreadsheetName, AccessPolicies) when is_list(AccessPolicies) ->
    case global:whereis_name(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};  % If the process is not found

        Pid when is_pid(Pid) ->
            %% Validate the access policies
            case validate_access_policies(AccessPolicies) of
                ok ->
                    %% Use gen_server:call to send the request synchronously
                    try
                        gen_server:call(Pid, {share, AccessPolicies})
                    catch
                        _:Error ->
                            io:format("Error encountered: ~p~n", [Error]),
                            {error, internal_error}
                    end;

                {error, Reason} ->
                    {error, Reason}
            end
    end.



Auxiliary Functions Refactoring
Let’s also clean up the auxiliary functions to improve their readability and maintainability.
validate_access_policies/1:
Recursive validation of each policy in the list.
Clear and concise error handling for invalid policies or malformed inputs.
validate_access_policy/1:
Ensures the policy is either read or write.
validate_proc/1:
Checks if the process is either a valid PID or a registered global process.
update_policies/2 handles the merging of new policies into the existing ones.


%% Handle the 'share' request in the gen_server
handle_call({share, NewPolicies}, From, State = #spreadsheet{owner = Owner, access_policies = CurrentPolicies}) ->
    %% Check if the calling process is the owner
    if 
        From =:= Owner ->
            %% Update the access policies
            UpdatedPolicies = update_policies(NewPolicies, CurrentPolicies),
            NewState = State#spreadsheet{access_policies = UpdatedPolicies},
            
            %% Log the updated policies (for debugging purposes)
            io:format("Updated access policies: ~p~n", [UpdatedPolicies]),
            
            %% Respond with success
            {reply, {ok, UpdatedPolicies}, NewState};
        
        true ->
            %% If not the owner, return an error
            {reply, {error, not_owner}, State}
    end.

Instead of sending a message directly with From ! {share_result, Result}, we now use gen_server:call/2's built-in reply mechanism ({reply, Result, State}).


# Move to Hints
Locally registered PIDs can be retrieved with whereis(RegisteredName).
Globally registered PIDs can be retrieved with global:whereis_name(GlobalName).
You can list all locally running processes on a node using erlang:processes/0.
To get the node of a specific PID, use erlang:node(Pid).


# Retrieve the Caller’s PID in handle_call/3 or handle_cast/2:

In gen_server:call/2, the PID of the calling process is passed to handle_call/3 as part of the From parameter.
You can retrieve the node and PID of the calling process using erlang:node(CallerPid) and element(1, From).
Access control logic can be implemented based on the calling process’s PID and node.
For asynchronous requests (gen_server:cast/2), you won’t receive the caller’s PID directly, but it’s better to use gen_server:call/2 when access control is needed.

# Example Workflow for Setting Up Distributed Nodes with a Shared Cookie
Set the Cookie: Ensure all nodes use the same cookie, either by:

Using the -setcookie flag when starting Erlang.
Setting the cookie in the .erlang.cookie file.
Connect Nodes: Use net_adm:ping(NodeName) to ensure the nodes can communicate.
Check Connected Nodes: You can verify the list of connected nodes by calling:

nodes().

## Troubleshooting Cookie Issues

Incorrect Cookie: If two nodes have different cookies, they won’t be able to connect, and you’ll see the pang result when using net_adm:ping/1.
File Permissions: Ensure the .erlang.cookie file has the correct permissions (read-only for the owner). If the file is not readable or has incorrect permissions, nodes won’t be able to connect.
Firewall/Network Issues: If nodes are on different machines, ensure that the appropriate ports (Erlang uses port 4369 for node discovery and a dynamic range for communication) are open for communication between nodes.



Introduce new module distributed_processess_utility to spawn and register processes in a distributed environment and make tests

refactor  reassign_owner/2

%% API Function: Reassign ownership of the spreadsheet
reassign_owner(SpreadsheetName, NewOwnerPid) when is_pid(NewOwnerPid) ->
    case global:whereis_name(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};  % The spreadsheet process doesn't exist
        Pid ->
            io:format("Trying to reassign owner to ~p~n", [NewOwnerPid]),
            %% Use gen_server:call/2 to synchronously send the request
            try
                gen_server:call(Pid, {reassign_owner, self(), NewOwnerPid})
            catch
                _:Error -> {error, internal_error}
            end
    end.

%% We need to update the spreadsheet process's logic to handle the %%reassign_owner request within the handle_call/3 callback. This %% %%will replace the original loop/1-based approach.
%% Handle the 'reassign_owner' request in the gen_server
handle_call({reassign_owner, NewOwner}, {FromPid, _Ref}, State = #spreadsheet{owner = CurrentOwner}) ->
    io:format("Ownership reassignment request from ~p to ~p~n", [CurrentOwner, NewOwner]),
    
    %% Check if the calling process (FromPid) is the current owner
    if
        FromPid =:= CurrentOwner ->
            %% Reassign the owner and update the state
            NewState = State#spreadsheet{owner = NewOwner},
            io:format("Ownership reassigned from ~p to ~p~n", [CurrentOwner, NewOwner]),
            {reply, {ok, NewOwner}, NewState};  %% Reply with success and new owner
        
        true ->
            %% Caller is not the current owner, return an error
            {reply, {error, unauthorized}, State}  %% No state change
    end.



## reassign_owner/2
API Function: We refactored reassign_owner/2 to use gen_server:call/2 for synchronous communication.
- Ownership Check: Corrected the check to ensure that only the current owner can reassign ownership.
- State Update: Updated the owner in the spreadsheet's state record.
- Error Handling: Provided appropriate responses for unauthorized attempts and invalid new owner PIDs.




In [ ]:
reassign_owner(SpreadsheetName, NewOwnerPid) when is_pid(NewOwnerPid) ->
    case global:whereis_name(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};  % The spreadsheet process doesn't exist
        Pid when is_pid(Pid) ->
            io:format("Attempting to reassign owner to ~p~n", [NewOwnerPid]),
            %% Use gen_server:call/2 for synchronous communication
            try
                gen_server:call(Pid, {reassign_owner, NewOwnerPid}, 5000)  %% 5 seconds timeout
            catch
                _:_Error -> {error, timeout}  %% Handle timeout or other errors
            end
    end.

In [ ]:
handle_call({reassign_owner, NewOwnerPid}, From, State = #spreadsheet{owner = Owner}) ->
    CallerPid = element(1, From),
    io:format("Reassign owner request from ~p (current owner: ~p) to new owner ~p~n", [CallerPid, Owner, NewOwnerPid]),
    %% Verify the calling process is the current owner
    case CallerPid of
        Owner ->
            %% Update the owner in the state
            NewState = State#spreadsheet{owner = NewOwnerPid},
            io:format("Ownership successfully reassigned from ~p to ~p~n", [Owner, NewOwnerPid]),
            {reply, ok, NewState};
        _ ->
            io:format("Unauthorized attempt to reassign ownership by ~p~n", [CallerPid]),
            {reply, {error, unauthorized}, State}
    end.


## Plan for Handling Ownership Reassignment After a Crash:
Handle Crashed Owner Shells:

If the shell of the current owner crashes, the ownership needs to be restored to the new shell (or a specific process that is the new shell for the same user).
restore_owner/2 API Function:

This function should allow restoring ownership in the special case where the current owner shell has crashed and is no longer alive. Only the process that was previously the owner (or possibly its new instance) should be able to restore ownership.
Ensure Authorization:

Ownership should never be changed by other processes on the same or different nodes, unless it’s the same user’s process restoring it after a crash.

In [ ]:
%% API function to restore the ownership of the spreadsheet if the current owner shell has crashed
restore_owner(SpreadsheetName, NewOwnerPid) when is_pid(NewOwnerPid) ->
    case global:whereis_name(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid when is_pid(Pid) ->
            %% Use gen_server:call to send a restore_owner request
            gen_server:call(Pid, {restore_owner, NewOwnerPid})
    end.


## Let's refactor the get/4 and get/5 functions and their associated logic into the gen_server framework. The main goals are to:

Handle the get requests within the gen_server callback using handle_call/3.
Retain the existing logic while improving the structure for OTP principles.
Steps for Refactoring:
API Functions (get/5, get/6): These functions will act as interfaces for clients to request a cell's value.
Access Control: We'll keep the check_access/3 logic intact to ensure only authorized processes can access cells.
Index Checking: We'll refactor the index boundary checks for rows, columns, and tabs into the gen_server logic.
Return Values: The cell value will be returned synchronously via gen_server:call/

In [ ]:
%% API function to get the value from a specific cell with default timeout (infinity)
get(SpreadsheetName, TabIndex, I, J) ->
    get(SpreadsheetName, TabIndex, I, J, infinity).

%% API function to get the value from a specific cell with a custom timeout
get(SpreadsheetName, TabIndex, I, J, Timeout) when is_integer(TabIndex), is_integer(I), is_integer(J), is_integer(Timeout) ->
    case global:whereis_name(SpreadsheetName) of
        undefined ->
            {error, spreadsheet_not_found};
        Pid when is_pid(Pid) ->
            %% Make a gen_server:call with a timeout
            try
                gen_server:call(Pid, {get, TabIndex, I, J}, Timeout)
            catch
                _:_ -> {error, timeout}
            end
    end.

%% Handle synchronous get requests
handle_call({get, Tab, I, J, Default}, From, State = #spreadsheet{tabs = Tabs, access_policies = Policies}) ->
    CallerPid = element(1, From),
    %% Check if the calling process has read access
    case check_access(CallerPid, Policies, read) of
        ok ->
            io:format("Received get request from ~p for Tab: ~p, Row: ~p, Col: ~p~n", [CallerPid, Tab, I, J]),
            %% Perform bounds checking
            case is_valid_indices(Tab, I, J, Tabs) of
                {ok, TabMatrix, Row} ->
                    %% Fetch the value from the spreadsheet
                    Value = lists:nth(J, Row),
                    {reply, {ok, Value}, State};
                {error, OutOfBounds} ->
                    io:format("Invalid access to Tab ~p, Row ~p, Col ~p~n", [Tab, I, J]),
                    {reply, {ok, Default}, State}
            end;
        {error, access_denied} ->
            io:format("Access denied for process ~p~n", [CallerPid]),
            {reply, {error, access_denied}, State}
    end.


%% Handle the 'get' request in the gen_server
handle_call({get, TabIndex, I, J}, _From, State = #spreadsheet{tabs = Tabs, access_policies = Policies}) ->
    %% Check if the calling process has read access
    case check_access(self(), Policies, read) of
        ok ->
            %% Ensure the TabIndex is within bounds
            if
                TabIndex > length(Tabs) orelse TabIndex < 1 ->
                    io:format("Tab index ~p is out of bounds~n", [TabIndex]),
                    {reply, {error, out_of_bounds}, State};
                true ->
                    %% Retrieve the Tab (TabMatrix) at TabIndex
                    TabMatrix = lists:nth(TabIndex, Tabs),
                    %% Ensure Row index I is within bounds
                    if
                        I > length(TabMatrix) orelse I < 1 ->
                            io:format("Row index ~p is out of bounds in Tab ~p~n", [I, TabIndex]),
                            {reply, {error, out_of_bounds}, State};
                        true ->
                            %% Retrieve the Row and ensure Column index J is within bounds
                            Row = lists:nth(I, TabMatrix),
                            if
                                J > length(Row) orelse J < 1 ->
                                    io:format("Col index ~p is out of bounds in Row ~p, Tab ~p~n", [J, I, TabIndex]),
                                    {reply, {error, out_of_bounds}, State};
                                true ->
                                    %% Retrieve the value at position (I, J)
                                    Value = lists:nth(J, Row),
                                    io:format("Returning value: ~p for Tab: ~p, Row: ~p, Col: ~p~n", [Value, TabIndex, I, J]),
                                    {reply, {ok, Value}, State}
                            end
                    end
            end;
        {error, access_denied} ->
            io:format("Access denied for process ~p~n", [self()]),
            {reply, {error, access_denied}, State}
    end.



Access Check: We first check if the calling process has read access using check_access/3.
Bounds Checking: We check if the TabIndex, I (row), and J (column) are within bounds. If any index is out of bounds, we return {error, out_of_bounds}.
Retrieve Value: If all indices are valid, we retrieve the value from the matrix at position (TabIndex, I, J) and return {ok, Value}.
Error Handling: If the caller doesn’t have read access, we return {error, access_denied}.

We can also simplify the find_registered_name/1 function, as searching through all registered() names could be costly in larger systems. If possible, we should prioritize using global registration (global:whereis_name/1).

In [ ]:
%% Find the registered name of a process based on its PID
find_registered_name(Pid) ->
    case lists:foldl(
        fun(Name, Acc) ->
            case global:whereis_name(Name) of
                Pid when Pid =/= undefined -> Name;
                _ -> Acc
            end
        end,
        undefined,
        registered()
    ) of
        undefined -> undefined;
        Name -> Name
    end.

## Private Helper Functions:

Helper functions like check_access/3 are not exported. These functions are purely internal and handle specific tasks, like validating access or manipulating the state.
Helper functions typically appear after the gen_server callbacks, as they are not part of the public API.
They can be made inline if performance is a concern, especially for frequently used helpers (as done with -compile({inline, [check_access/3]}) in the example).
Clear Separation of Concerns:

Keeping the helper functions separated and private allows for a clean structure where public API functions, gen_server callback logic, and internal helpers are all clearly organized.
Key Considerations for Helper Functions:
Naming and Purpose:

Keep helper functions focused on specific tasks (e.g., access control, state updates, validation). Name them according to their role (e.g., check_access/3, find_registered_name/1, handle_get/4).
Inline Compilation:

For performance reasons, you may wish to mark small, frequently used helper functions as inline using -compile({inline, [check_access/3, find_registered_name/1]}). This tells the compiler to inline the function calls, reducing function call overhead.
Encapsulation:

Helper functions like check_access/3 and find_registered_name/1 should not be directly exposed to the outside world. They should only be called within the module where they’re defined, ensuring proper encapsulation of logic.

## Let's refactor set/5 and set/6 functions.

Here's the code for set/5 and its extension set/6 with Timeout, then the logic to implement to handle request
and finally auxiliary function


In [ ]:

set(SpreadsheetName, TabIndex, I, J, Value, Timeout) ->
    case global:whereis_name(SpreadsheetName) of
        undefined -> {error, spreadsheet_not_found};
        Pid ->
           % Send the get request to the spreadsheet process  
                %ERRATO
                %Prima di inviare la richiesta di set al processo del foglio di calcolo, controlliamo se il processo %chiamante (self()) ha accesso in scrittura. In caso contrario, restituiamo {error, access_denied}.
                %case check_access(self(), SpreadsheetName#spreadsheet.access_policies, write) of
                %ok ->

                    Pid ! {set, self(), TabIndex, I, J, Value},
                    receive
                        {set_result, ok} -> ok;
                        {error, access_denied} -> {error, access_denied}
                    after Timeout ->
                         {error, timeout}
                    end
                %end
    end.
    % Here the logic to implement to handle request
    % Handle set request
        {set, From, Tab, I, J, Val} ->
        % Check if the requesting process has write access
            case check_access(From, Policies, write) of
                ok ->

                    io:format("Received set request for Tab: ~p, Row: ~p, Col: ~p, Val: ~p~n", [Tab, I, J, Val]),
                    if
                        Tab > length(Tabs) orelse Tab < 1 ->
                            io:format("Tab index ~p is out of bounds~n", [Tab]),
                            From ! {set_result, false};
                        true ->
                            TabMatrix = lists:nth(Tab, Tabs),
                            if
                                I > length(TabMatrix) orelse I < 1 ->
                                    io:format("Row index ~p is out of bounds in Tab ~p~n", [I, Tab]),
                                    From ! {set_result, false};
                                true ->
                                    Row = lists:nth(I, TabMatrix),
                                    if
                                        J > length(Row) orelse J < 1 ->
                                            io:format("Col index ~p is out of bounds in Row ~p, Tab ~p~n", [J, I, Tab]),
                                            From ! {set_result, false};
                                        true ->
                                            io:format("Setting value ~p at Tab: ~p, Row: ~p, Col: ~p~n", [Val, Tab, I, J]),
                                            NewRow = replace_nth(J, Val, Row),
                                            NewTabMatrix = replace_nth(I, NewRow, TabMatrix),
                                            NewTabs = replace_nth(Tab, NewTabMatrix, Tabs),
                                            % Update the last modified timestamp
                                            CurrentTime = calendar:universal_time(),
                                            NewState = State#spreadsheet{tabs = NewTabs, last_modified = CurrentTime},
                                            From ! {set_result, true},
                                            loop(NewState)
                                    end
                            end
                    end;
                {error, access_denied} ->
                    From ! {error, access_denied},
                    loop(State)
            end;                  
%and finally auxiliary function
%Funzione ausiliaria per rimpiazzare un elemento in una lista al valore di indice dato
replace_nth(Index, NewVal, List) ->
    {Left, [_|Right]} = lists:split(Index-1, List),
    Left ++ [NewVal] ++ Right.

In the refactor solution that will implement gen_server scheme, we must keep in considertion theat Value should have to of every basic type provided from erlang language

MUOVI SU

# Let's refactor the set/5 and set/6 functions to follow the gen_server scheme for the distributed_spreadsheet module.

Key Objectives:
Synchronous Communication: Use gen_server:call/2 instead of sending a message with Pid ! ... for the set request.
Timeout Handling: Use gen_server:call/3 for managing timeouts in the set/6 function.
Access Control: Ensure that the caller has write access before making changes to the spreadsheet.
Refactoring Logic: Use helper functions for modularity (like replace_nth/3), and integrate the timestamp updates.

## Refine check_access/3

Superior Write Access Check:

We first check if the resolved PID (or registered name) has write access.
If write access is found, we immediately return ok to the caller, as having write access also implies they have superior access to perform read operations.
Check for Read Access:

If no write access is found, we then check if the caller has read access.
If read access is found and the caller requested read access, we return ok.
Final Case:

If neither write nor read access is found, we return {error, access_denied}.

In [ ]:
%% Check if the calling process has the required access (read/write)

check_access(PidOrName, Policies, RequiredAccess) ->
    io:format("Checking if ~p process has ~p access in policies: ~p~n", [PidOrName, RequiredAccess, Policies]),

    %% Resolve PidOrName to both PID and registered name if possible
    ResolvedPid = case is_pid(PidOrName) of
                     true -> PidOrName;
                     false -> global:whereis_name(PidOrName)
                  end,

    %% Check if the resolved PID or registered name has the 'write' access
    case lists:keyfind(ResolvedPid, 1, Policies) of
        {ResolvedPid, write} ->
            io:format("Access granted with superior write access for PID: ~p~n", [ResolvedPid]),
            ok;  %% If write access is found, return ok immediately
        _ ->
            %% Check for access by registered name (if available)
            case find_registered_name(ResolvedPid) of
                undefined -> check_read_access(PidOrName, Policies, RequiredAccess);
                RegisteredName ->
                    case lists:keyfind(RegisteredName, 1, Policies) of
                        {RegisteredName, write} ->
                            io:format("Access granted with superior write access for registered name: ~p~n", [RegisteredName]),
                            ok;  %% If write access is found by name, return ok immediately
                        _ -> check_read_access(RegisteredName, Policies, RequiredAccess)
                    end
           end
    end. 

%% Helper function to check for read access
check_read_access(PidOrName, Policies, RequiredAccess) ->
    case lists:keyfind(PidOrName, 1, Policies) of
        {PidOrName, read} when RequiredAccess == read ->
            io:format("Read access granted for ~p~n", [PidOrName]),
            ok;
        _ -> {error, access_denied}
    end.


## Let's proceed with api functions to_csv/2 and to_csv/3.

As usual I pass you the code for refactor(to_csv/3 implements only Timeout moreover) into gen_server scheme.Take care of basic possible type values into cells  checked by is_basic_type/1

## To refactor the from_csv/1 function into a gen_server-compatible scheme..
we'll keep in mind the need to handle the various data types supported in the cells of the spreadsheet. We also need to ensure that the metadata (spreadsheet name, owner, access policies, etc.) is correctly retrieved and parsed from the CSV file.

Key Refactor Objectives:
The from_csv/1 function reads the SpreadsheetName from the CSV and dynamically determines whether to start a new spreadsheet process or update an existing one.
The gen_server process manages the state update and ensures that the spreadsheet state is updated with the newly loaded data from the CSV file.
This design allows the CSV file to act as a backup or persistence mechanism for the spreadsheet process, fitting neatly into the gen_server model.
Handle All Basic Erlang Types: We'll make sure the from_csv/1 function correctly parses the cell data and recognizes the basic Erlang types we support.
Use gen_server for State: We'll retrieve the state of the spreadsheet and update it with the newly loaded data from the CSV.
Error Handling: We'll ensure proper error handling for issues like file not found, invalid formatting, and parsing errors.
Parsing Metadata and Tabs: We'll focus on correctly parsing the spreadsheet metadata (like the owner, access policies, etc.) and then loading the tabs with proper type conversion for the cells.

The parse_csv/1 function is responsible for reading and parsing the entire CSV file, extracting the metadata (spreadsheet name, owner, access policies, etc.), and loading the tabs as well.
We need to load the tabs (the matrix data) from the CSV file. This is done row-by-row, where each row represents a row in the tab.load_tabs_from_csv/1
For each row, we need to parse each cell and interpret it based on its content.
We need to convert certain strings back into Erlang terms, such as the access policies, which were serialized as strings.

Helper Function parse_term_from_string/



Problem:
The format <0.90.0> is a PID in Erlang, and when written as part of the CSV file, it's stored as a string. However, the parsing tools like erl_scan:string/1 expect valid Erlang syntax, and a PID in the form <0.90.0> is not something Erlang can directly interpret from a string.

Solution:
We need to treat PIDs differently during both writing to the CSV and parsing from the CSV. Specifically, we'll:

Convert PIDs to a string when saving the spreadsheet to a CSV.
Parse the string back into a PID when loading the spreadsheet from a CSV file.

In [ ]:
# Set Up the OTP Supervisor

-module(distributed_spreadsheet_sup).
-behaviour(supervisor).

-export([start_link/0, init/1]).

%% Start the supervisor
start_link() ->
    supervisor:start_link({global, distributed_spreadsheet_sup}, ?MODULE, []).

%% Initialize the supervisor
init([]) ->
    %% Define child processes to be supervised
    SpreadsheetSpec = {
        distributed_spreadsheet, %% The child process (our gen_server)
        {distributed_spreadsheet, start_link, []}, %% Start function
        permanent, %% If the process dies, always restart it
        5000, %% Time to wait before restarting
        worker, %% The type of process (worker, not supervisor)
        [distributed_spreadsheet] %% The module
    },

    %% Strategy: one-for-one, if one child dies, restart only that child
    {ok, {{one_for_one, 10, 10}, [SpreadsheetSpec]}}.`


Explanation:
distributed_spreadsheet_sup: The supervisor that starts and monitors your distributed_spreadsheet process.
one_for_one strategy: If the spreadsheet process fails, only that process will be restarted.
permanent restart: The spreadsheet process will always be restarted if it terminates, regardless of the reason.
You can extend this to monitor multiple spreadsheet processes, each handling different instances of spreadsheets if needed.

## integrate the global registration of the owner process into the logic of both new/1 and new/4 functions to ensure that the global registration and monitoring of the owner begin as soon as the spreadsheet is created.



In [ ]:
new(SpreadsheetName, N, M, K) when is_integer(N), is_integer(M), is_integer(K), N > 0, M > 0, K > 0 ->
    case global:whereis_name(SpreadsheetName) of
        undefined ->
            %% Spreadsheet doesn't exist, create it
            LastModified = erlang:system_time(),
            OwnerPid = self(),  % The shell that created the spreadsheet is the owner
            Result = gen_server:start_link({global, SpreadsheetName}, ?MODULE, {SpreadsheetName, OwnerPid, N, M, K, LastModified}, []),
            
            %% Now register the owner globally and monitor the process
            case Result of
                {ok, Pid} ->
                    register_owner(SpreadsheetName, OwnerPid),
                    {ok, Pid};
                Error -> Error
            end;
        _ ->
            {error, already_exists}
    end;
new(_, _, _, _) -> {error, invalid_parameters}.


You can encapsulate the logic for registering and monitoring the owner in a dedicated function called register_owner/2 as mentioned earlier. This function will be called whenever a new spreadsheet is created (both in new/1 and new/4).

erlang

%% Register the owner globally and monitor the owner process
register_owner(SpreadsheetName, OwnerPid) ->
    %% Register the owner globally
    case global:register_name({SpreadsheetName, owner}, OwnerPid) of
        ok ->
            io:format("Owner ~p registered globally for spreadsheet ~p~n", [OwnerPid, SpreadsheetName]),
            %% Monitor the owner process
            erlang:monitor(process, OwnerPid),
            ok;
        {error, Reason} ->
            io:format("Failed to register owner for spreadsheet ~p: ~p~n", [SpreadsheetName, Reason]),
            {error, Reason}
    end.
Explanation:
new/1 delegates to new/4 by providing default values for N, M, and K (for rows, columns, and tabs).
new/4:
After starting the gen_server process for the spreadsheet, the function registers the owner (shell process) globally using global:register_name/2.
It also sets up monitoring using erlang:monitor/2 to track the owner PID.
If the spreadsheet already exists, it returns an {error, already_exists}.
register_owner/2 is the function responsible for globally registering the owner PID and monitoring the process for failure

Restoring the Owner If It Changes
If the shell restarts (or moves to another node), you can use the restore_owner/2 function to re-register the new shell’s PID.

erlang
Copia codice
restore_owner(SpreadsheetName) ->
    NewOwnerPid = self(),  % The new shell is the new owner
    case global:re_register_name({SpreadsheetName, owner}, NewOwnerPid) of
        ok ->
            io:format("Ownership of spreadsheet ~p restored to ~p~n", [SpreadsheetName, NewOwnerPid]),
            erlang:monitor(process, NewOwnerPid),
            {ok, NewOwnerPid};
        {error, Reason} ->
            io:format("Failed to restore ownership for spreadsheet ~p: ~p~n", [SpreadsheetName, Reason]),
            {error, Reason}
    end.

Handling Owner Process Failure
To handle the failure of the owner (shell) process, the gen_server can use the handle_info/2 function to catch the DOWN message when the owner process dies. You may already have this logic from the previous step.

erlang
Copia codice
handle_info({'DOWN', _Ref, process, OwnerPid, _Reason}, State) ->
    io:format("Owner process ~p has crashed. Waiting for a new shell to take ownership...~n", [OwnerPid]),
    %% The owner process is down, you can trigger manual recovery here
    {noreply, State};
    
handle_info(_Msg, State) ->
    {noreply, State}.
This will print a message when the owner process dies, and you can choose to take recovery actions (such as waiting for a new owner).



Summary:
Supervision:

The supervisor ensures that if the distributed_spreadsheet process crashes, it will be automatically restarted.
This ensures that the spreadsheet process is resilient and can recover from failures.
Monitoring the Shell:

The gen_server monitors the shell process (owner), so if the shell crashes, it will receive a DOWN message and can take action.
If both the shell and the distributed_spreadsheet crash, you can restore the state and ownership after restart.
State Restoration:

After a crash, the distributed_spreadsheet can reload its state from a CSV file if needed, ensuring that it can recover the last known state.
Restoring Ownership:

When the shell (or a new shell) restarts, you can use restore_owner/2 to re-register the shell as the owner of the spreadsheet and continue monitoring the new owner.

## Key Scenarios to Handle in init/1:
Fresh Initialization: This happens when you call new/1 or new/4. You initialize a new spreadsheet with fresh tabs, owner, etc.
Restoration from CSV: If the system crashes and is restarted, you want the gen_server to reload its state from a CSV file automatically, without needing manual intervention.
Restoration from from_csv/1: This happens when you explicitly load a spreadsheet from a CSV file using the from_csv/1 command.

### Step 3: Recovery from CSV in Case of Total Failure( da implementare)
If both the process and its owner crash (or if the node itself crashes), you can recover by reloading the state from the CSV file.

In the supervisor or in a higher-level manager, you can use the from_csv/1 function to recover the spreadsheet when it detects a critical failure.

Example Recovery Logic:
erlang
Copia codice
recover_spreadsheet(SpreadsheetName, CSVFile) ->
    case distributed_spreadsheet:from_csv(CSVFile) of
        {ok, Pid} ->
            io:format("Spreadsheet ~p successfully recovered from CSV~n", [SpreadsheetName]),
            {ok, Pid};
        {error, Reason} ->
            io:format("Failed to recover spreadsheet ~p: ~p~n", [SpreadsheetName, Reason]),
            {error, Reason}
    end.
You can call this recovery function when the supervis

## Monitor Nodes in distributed_spreadsheet_sup (Supervisor)
If your goal is to monitor nodes for overall cluster availability and ensure the system can recover if a node goes down, putting net_kernel:monitor_nodes/1 in the supervisor process might be a good approach.

In this case, the supervisor is responsible for monitoring node failures and restarting or redistributing responsibilities across nodes when necessary. It can take global actions when a node goes down, such as restarting processes, shifting ownership, or recovering from failure.

Why Monitor in Supervisor:
Node-level Monitoring: The supervisor oversees multiple processes and can manage failover for node-level events (node up/down). It can handle restarting all gen_server processes in case of a node failure or initiating a recovery plan when nodes fail.
Better Failover Strategy: If a node goes down, the supervisor can ensure that processes on that node are restarted on another available node. It’s a good place for system-level actions like shifting work to other nodes or ensuring that the system stays running.

## Running a Supervisor on Multiple Nodes (Distributed Supervisors)

Instead of running the supervisor only on node1, you can set up multiple supervisors on different nodes (e.g., node2, node3) that can take over the supervision responsibility if node1 goes down.

This is known as distributed supervision, where multiple nodes have a supervisor that manages the processes and can take over when another node’s supervisor fails.

How Distributed Supervisors Work:
Each node in the cluster has a local supervisor that monitors the local processes (proc1, proc2, etc.) and the distributed_spreadsheet process.
When a node (e.g., node1) fails, another node’s supervisor (e.g., on node2) can detect the failure, restart the necessary processes, and potentially transfer the ownership of the distributed_spreadsheet.
Key Points:
net_kernel:monitor_nodes(true) in init/1:

This enables the supervisor to monitor node events (nodes joining or leaving the cluster).
It starts monitoring when the supervisor is initialized.
Handling nodeup and nodedown Messages:

nodeup: When a node joins the cluster, you log the event but take no other action.
nodedown: When a node leaves the cluster, you check if it was node1 (where proc1 and distributed_spreadsheet were running). If so, it transfers ownership of the distributed_spreadsheet process to another node using transfer_ownership/1.
Ownership Transfer:

The transfer_ownership/1 function is called if node1 fails. This function can be used to transfer ownership of the distributed_spreadsheet process to another available node. You should implement this function according to your existing ownership transfer logic.
Stopping Node Monitoring in terminate/2:

When the supervisor terminates, node monitoring is stopped by calling net_kernel:monitor_nodes(false).

